Importing important libraries:

In [257]:
import pandas
import numpy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

Acquiring dataset and making a dataframe:

In [258]:
weather = pandas.read_csv("/Users/anay/Desktop/RoboManipal/RoboManipal_Anay/Data_Preprocessing/weatherAUS.csv")
weatherdf = pandas.DataFrame(weather)

Getting basic overview of the data:

In [259]:
print(weather.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

I am first going to only focus on filling NaN values in int64 and float64 type columns. According to the kaggle article, we need to drop all the columns where there is more than 75% missing data, we do not have any such columns so we will instead try to fill them up.

In order to fill the missing values, I will first see the different ranges of values present in each column.

In [260]:
less_missing_columns_intfloat = ["MinTemp", "MaxTemp", "Rainfall", "WindGustSpeed", "WindSpeed9am", "WindSpeed3pm", "Humidity9am", "Humidity3pm", "Pressure9am", "Pressure3pm", "Temp9am", "Temp3pm"]

for i in less_missing_columns_intfloat:
    print(i)
    print(weatherdf[i].describe())

MinTemp
count    143975.000000
mean         12.194034
std           6.398495
min          -8.500000
25%           7.600000
50%          12.000000
75%          16.900000
max          33.900000
Name: MinTemp, dtype: float64
MaxTemp
count    144199.000000
mean         23.221348
std           7.119049
min          -4.800000
25%          17.900000
50%          22.600000
75%          28.200000
max          48.100000
Name: MaxTemp, dtype: float64
Rainfall
count    142199.000000
mean          2.360918
std           8.478060
min           0.000000
25%           0.000000
50%           0.000000
75%           0.800000
max         371.000000
Name: Rainfall, dtype: float64
WindGustSpeed
count    135197.000000
mean         40.035230
std          13.607062
min           6.000000
25%          31.000000
50%          39.000000
75%          48.000000
max         135.000000
Name: WindGustSpeed, dtype: float64
WindSpeed9am
count    143693.000000
mean         14.043426
std           8.915375
min           0.

Now lets find missing values

In [261]:
weatherdf.isnull().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

ok, now i will find which of these columns i will need to drop, i think 1.4 lakh are significant enough amount of samples to not have bias, so whatever column has 75% of its values missing will be dropped.

In [262]:
drop_columns = ["Evaporation", "Sunshine", "Cloud9am", "Cloud3pm"]        
weatherdf = weatherdf.drop(drop_columns, axis=1)
weatherdf.isnull().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [263]:
print(weatherdf.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   WindGustDir    135134 non-null  object 
 6   WindGustSpeed  135197 non-null  float64
 7   WindDir9am     134894 non-null  object 
 8   WindDir3pm     141232 non-null  object 
 9   WindSpeed9am   143693 non-null  float64
 10  WindSpeed3pm   142398 non-null  float64
 11  Humidity9am    142806 non-null  float64
 12  Humidity3pm    140953 non-null  float64
 13  Pressure9am    130395 non-null  float64
 14  Pressure3pm    130432 non-null  float64
 15  Temp9am        143693 non-null  float64
 16  Temp3pm        141851 non-null  float64
 17  RainToday      142199 non-nul

So now, i will remove the outliers.

In [264]:
weatherdf["Date"] = pandas.to_datetime(weatherdf["Date"], errors='coerce')

numeric = weatherdf.select_dtypes(include=[numpy.number])
iqr = weatherdf.quantile(0.75, numeric_only=True) - weatherdf.quantile(0.25, numeric_only=True)
lim = numpy.abs((numeric - numeric.median()) / iqr) < 1.5
weatherdf[numeric.columns] = numeric.where(lim, numpy.nan)

Now, finally lets fill the nans.

In [265]:
for i in numeric.columns:
    weatherdf[i].fillna(weatherdf[i].mean(), inplace=True)

/var/folders/mf/dh_37v4d2d9_9g1gc7qfmm740000gn/T/ipykernel_925/2592011918.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  weatherdf[i].fillna(weatherdf[i].mean(), inplace=True)


So i get the above error every time i try this. now i am just going to ignore it bc stack overflow says that it gets triggered as chain assignment when i am just overwriting values in a column. 

itll be good practice to check how many nans remain now.

In [266]:
weatherdf.isnull().sum()

Date                 0
Location             0
MinTemp              0
MaxTemp              0
Rainfall             0
WindGustDir      10326
WindGustSpeed        0
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am         0
WindSpeed3pm         0
Humidity9am          0
Humidity3pm          0
Pressure9am          0
Pressure3pm          0
Temp9am              0
Temp3pm              0
RainToday         3261
RainTomorrow      3267
dtype: int64

ok, humidity for some reason isnt getting updated here, so ill just do that manually.

In [267]:
weatherdf["Humidity9am"].fillna(weatherdf["Humidity9am"].mean(), inplace=True)
weatherdf["Humidity3pm"].fillna(weatherdf["Humidity3pm"].mean(), inplace=True)
weatherdf.isnull().sum()

/var/folders/mf/dh_37v4d2d9_9g1gc7qfmm740000gn/T/ipykernel_925/2592692861.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  weatherdf["Humidity9am"].fillna(weatherdf["Humidity9am"].mean(), inplace=True)
/var/folders/mf/dh_37v4d2d9_9g1gc7qfmm740000gn/T/ipykernel_925/2592692861.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which 

Date                 0
Location             0
MinTemp              0
MaxTemp              0
Rainfall             0
WindGustDir      10326
WindGustSpeed        0
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am         0
WindSpeed3pm         0
Humidity9am          0
Humidity3pm          0
Pressure9am          0
Pressure3pm          0
Temp9am              0
Temp3pm              0
RainToday         3261
RainTomorrow      3267
dtype: int64

FINALLY. now on to the next step that is encoding the data. the kaggle article is suggesting using functions from scikitlearn, but im told to avoid that, so ill be using the get_dummies for this.
actually before i do that, i should convert the date column to a datetime type.

In [268]:
weatherdf["Date"] = pandas.to_datetime(weatherdf["Date"], format='%Y%m%d.0', errors='coerce')
weatherdf["Date"].isnull().sum()

np.int64(0)

ok so no bad values thats good, now filling out all the objects

In [269]:
obj = weatherdf.select_dtypes(include=['object']).drop(columns=['Location'])
for i in obj.columns:
    weatherdf[i].fillna(weatherdf[i].mode().iloc[0], inplace=True)

/var/folders/mf/dh_37v4d2d9_9g1gc7qfmm740000gn/T/ipykernel_925/368804801.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  weatherdf[i].fillna(weatherdf[i].mode().iloc[0], inplace=True)


Checking for remaining nulls:

In [270]:
weatherdf.isnull().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

Now finally getting dummies:

In [271]:
weatherdf = pandas.get_dummies(weatherdf, columns=obj.columns)
weatherdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 66 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Date              145460 non-null  datetime64[ns]
 1   Location          145460 non-null  object        
 2   MinTemp           145460 non-null  float64       
 3   MaxTemp           145460 non-null  float64       
 4   Rainfall          145460 non-null  float64       
 5   WindGustSpeed     145460 non-null  float64       
 6   WindSpeed9am      145460 non-null  float64       
 7   WindSpeed3pm      145460 non-null  float64       
 8   Humidity9am       145460 non-null  float64       
 9   Humidity3pm       145460 non-null  float64       
 10  Pressure9am       145460 non-null  float64       
 11  Pressure3pm       145460 non-null  float64       
 12  Temp9am           145460 non-null  float64       
 13  Temp3pm           145460 non-null  float64       
 14  Wind

Now, i will be splitting the dataset:

In [272]:
X = weatherdf
y = weatherdf

X_train, X_test, y_train, y_split = train_test_split(X, y, test_size=0.2, random_state=42) 

Finally, I will be scaling the dataset. i am over here going to do minmax scaling since i have removed the outliers already.

In [273]:
scale_columns = X_train.columns[2:]

mm = MinMaxScaler()

X_train[scale_columns] = mm.fit_transform(X_train[scale_columns])
X_test[scale_columns] = mm.fit_transform(X_test[scale_columns])

Exporting the dataframe to a new file:

In [274]:
X_train.to_csv("/Users/anay/Desktop/RoboManipal/RoboManipal_Anay/Data_Preprocessing/weatherAUS_train.csv", index=False)
X_test.to_csv("/Users/anay/Desktop/RoboManipal/RoboManipal_Anay/Data_Preprocessing/weatherAUS_test.csv", index=False)